# 🚦 YOLO Detection Server V16 - VIOLATION DETECTION
### NEW: Violation Detection + License Plate → /ws/kaggle
### Features:
- Red Light Violation (vượt đèn đỏ)
- Lane Encroachment (lấn làn)
- License Plate Recognition
- New WebSocket path: /ws/kaggle

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Cell 1: CONFIGURATION
# ═══════════════════════════════════════════════════════════════

# Backend domain (chung cho HTTP va WS)
BACKEND_URL = 'https://your-backend.trycloudflare.com'
NMS_URL = 'https://your-nms.trycloudflare.com'

# Auto-generate URLs
BACKEND_HTTP_URL = BACKEND_URL
BACKEND_WS_URL = BACKEND_URL.replace('https://', 'wss://').replace('http://', 'ws://') + '/ws/kaggle'

# Detection settings
TRACK_LINE_Y = 50  # Counting line Y position (%)
CONFIDENCE = 0.5
IMGSZ = 640
USE_HALF = True

# Queue sizes
FRAME_QUEUE_SIZE = 3
RESULT_QUEUE_SIZE = 10

# Vehicle classes
VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']

# Camera API keys (loaded from backend)
camera_keys = {}

print(f"📡 HTTP: {BACKEND_HTTP_URL}")
print(f"📡 WS: {BACKEND_WS_URL}")
print(f"📺 NMS: {NMS_URL}")

In [ ]:
# Cell 2: Install Dependencies
!pip uninstall -y numpy pillow ultralytics > /dev/null 2>&1
!pip install "numpy<2.0.0" "pillow>=10.3.0" scipy ultralytics opencv-python-headless requests websocket-client --upgrade --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

import numpy, PIL
print(f'✅ Dependencies installed. Numpy: {numpy.__version__}, PIL: {PIL.__version__}')

In [ ]:
# Cell 3: Load Models
import warnings
warnings.filterwarnings('ignore')
import os, torch
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🚀 Device: {device}')

vehicle_model = traffic_light_model = lp_detector = char_detector = None
base_dir = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/2/'

# Vehicle Detection
try:
    vehicle_model = YOLO('yolo11m.pt').to(device)
    print('✅ Vehicle Model')
except Exception as e: print(f'⚠️ Vehicle: {e}')

# Traffic Light Detection
try:
    p = os.path.join(base_dir, 'mhiot-dentinhieu-best-new.pt')
    if os.path.exists(p):
        traffic_light_model = YOLO(p).to(device)
        print('✅ Traffic Light Model')
    else: print(f'⚠️ TL Model not found')
except Exception as e: print(f'⚠️ TL: {e}')

# License Plate Detection
try:
    p = os.path.join(base_dir, 'license_plate_detector.pt')
    if os.path.exists(p):
        lp_detector = YOLO(p).to(device)
        print('✅ LP Detector')
except Exception as e: print(f'⚠️ LP: {e}')

# Character Detection for OCR
try:
    p = os.path.join(base_dir, 'char_detector.pt')
    if os.path.exists(p):
        char_detector = YOLO(p).to(device)
        print('✅ Char Detector')
except Exception as e: print(f'⚠️ Char: {e}')

In [ ]:
# Cell 4: Detection Functions + VIOLATION DETECTION
import cv2, numpy as np, time, requests, threading, re, queue, json
from datetime import datetime
from collections import deque

# Camera state
camera_trackers = {}
camera_configs = {}  # Store lane configs from backend
current_traffic_light = {}  # Store current traffic light status per camera

def log(msg):
    print(f'[{datetime.now().strftime("%H:%M:%S")}] {msg}')

def read_plate_characters(plate_crop):
    """OCR for Vietnamese license plate"""
    try:
        if char_detector is None: return 'unknown'
        h, w = plate_crop.shape[:2]
        if h < 10 or w < 20: return 'unknown'
        results = char_detector(plate_crop, conf=0.3, verbose=False)
        chars = []
        for b in results[0].boxes:
            x1 = float(b.xyxy[0][0])
            cls_id = int(b.cls[0])
            char_val = results[0].names[cls_id]
            chars.append((x1, char_val))
        if len(chars) < 5: return 'unknown'
        chars.sort(key=lambda x: x[0])
        text = ''.join([c[1] for c in chars])
        return text.upper().replace(' ', '').replace('-', '')
    except: return 'unknown'

def detect_license_plate(frame, x1, y1, x2, y2):
    """Detect license plate in vehicle bounding box"""
    if lp_detector is None: return None
    try:
        vehicle_crop = frame[y1:y2, x1:x2]
        results = lp_detector(vehicle_crop, conf=0.4, verbose=False)
        if len(results[0].boxes) == 0: return None
        max_conf = 0; best_plate = None
        for b in results[0].boxes:
            conf = float(b.conf[0])
            if conf > max_conf:
                max_conf = conf
                px1, py1, px2, py2 = map(int, b.xyxy[0])
                plate_crop = vehicle_crop[py1:py2, px1:px2]
                text = read_plate_characters(plate_crop)
                if text != 'unknown':
                    best_plate = {'text': text, 'confidence': max_conf}
        return best_plate
    except: return None

def check_lane_violation(vehicle_class, center_x, camera_id, img_width):
    """
    Check if vehicle is in wrong lane
    Returns: True if violation, False otherwise
    """
    config = camera_configs.get(camera_id)
    if not config: return False
    
    lane_points = config.get('camera_lane_track_point', [])
    lane_vehicles = config.get('camera_lane_vehicles', [])
    
    if not lane_points or not lane_vehicles: return False
    
    # Determine which lane the vehicle is in
    x_percent = (center_x / img_width) * 100
    boundaries = [0] + sorted(lane_points) + [100]
    
    for i in range(len(boundaries) - 1):
        if boundaries[i] <= x_percent < boundaries[i + 1]:
            # Found the lane, check if vehicle type is allowed
            if i < len(lane_vehicles):
                allowed = lane_vehicles[i]
                if isinstance(allowed, list):
                    # ANY means all vehicles allowed
                    if 'ANY' in [v.upper() for v in allowed]:
                        return False
                    if vehicle_class.lower() not in [v.lower() for v in allowed]:
                        return True  # VIOLATION!
            break
    return False

def check_red_light_violation(track_id, prev_y, curr_y, camera_id, img_height):
    """
    Check if vehicle crossed counting line while light is RED
    Returns: True if violation, False otherwise
    """
    tl_status = current_traffic_light.get(camera_id, 'UNKNOWN')
    if tl_status != 'RED': return False
    
    line_y = (TRACK_LINE_Y / 100) * img_height
    
    # Check if vehicle crossed the line (from above to below)
    if prev_y is not None and prev_y < line_y <= curr_y:
        return True
    return False

In [ ]:
# Cell 5: Main Detection Function V17 (ALWAYS DETECT PLATES)\n\ndef detect_frame(frame, camera_id):\n    """Process a single frame and return detection results with violations"""\n    global current_traffic_light\n    \n    h, w = frame.shape[:2]\n    result = {\n        'camera_id': camera_id,\n        'created_at': int(time.time() * 1000),\n        'image_dimensions': {'width': w, 'height': h},\n        'detections': [],\n        'vehicle_count': 0,\n        'tracks': [],\n        'license_plates': [], # Store ALL detected plates\n        'violations': []\n    }\n    \n    # Initialize tracker for camera\n    if camera_id not in camera_trackers:\n        camera_trackers[camera_id] = BYTETracker(TRACKER_ARGS)\n    tracker = camera_trackers[camera_id]\n    \n    try:\n        # Vehicle Detection\n        model_result = vehicle_model(frame, classes=[2, 3, 5, 7], conf=0.3, verbose=False)[0]\n        
        # Prepare detections for tracker\n        detections_list = []\n        for box in model_result.boxes:\n            x1, y1, x2, y2 = map(int, box.xyxy[0])\n            conf = float(box.conf[0])\n            cls = int(box.cls[0])\n            detections_list.append([x1, y1, x2, y2, conf, cls])\n        \n        detections_array = np.array(detections_list) if detections_list else np.empty((0, 6))\n        tracks = tracker.update(detections_array, (h, w), (h, w))\n        \n        final_detections = []\n        \n        for t in tracks:\n            tlbr = t.tlbr\n            x1, y1, x2, y2 = map(int, tlbr)\n            track_id = t.track_id\n            cls_id = t.class_id\n            cls_name = model_result.names[cls_id] if cls_id < len(model_result.names) else str(cls_id)\n            \n            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2\n            \n            # 1. LICENSE PLATE DETECTION (ALWAYS)\n            # Only run if vehicle area is large enough (>0.5% image)\n            lp_text = 'unknown'\n            lp_conf = 0.0\n            area = (x2-x1)*(y2-y1)\n            if area > (w*h)*0.005:\n                lp_info = detect_license_plate(frame, x1, y1, x2, y2)\n                if lp_info and lp_info['text'] != 'unknown':\n                    lp_text = lp_info['text']\n                    lp_conf = lp_info['confidence']\n                    # Add to generic list\n                    result['license_plates'].append({\n                        'plate': lp_text,\n                        'vehicle_id': track_id,\n                        'confidence': lp_conf\n                    })\n            \n            det_obj = {\n                'bbox': [x1, y1, x2, y2],\n                'class': cls_name,\n                'id': track_id,\n                'license_plate': lp_text\n            }\n            final_detections.append(det_obj)\n            \n            # 2. TRACK HISTORY & VIOLATIONS\n            if track_id not in track_history:\n                track_history[track_id] = {'positions': [], 'prev_positions': {}}\n            \n            tr = track_history[track_id]\n            if camera_id not in tr['prev_positions']: tr['prev_positions'][camera_id] = None\n            prev_y = tr['prev_positions'][camera_id]\n            \n            # Check RED LIGHT\n            if check_red_light_violation(track_id, prev_y, cy, camera_id, h):\n                result['violations'].append({\n                    'type': 'RED_LIGHT',\n                    'license_plate': lp_text,\n                    'confidence': lp_conf,\n                    'bbox': [x1, y1, x2, y2],\n                    'detection_id': track_id,\n                    'image_crop': None # Optimization: don't send blob for now unless needed\n                })\n                log(f'🚨 RED LIGHT: {lp_text}')\n            \n            # Check LANE\n            if check_lane_violation(cls_name, cx, camera_id, w):\n                 result['violations'].append({\n                    'type': 'LANE_ENCROACHMENT',\n                    'license_plate': lp_text,\n                    'confidence': lp_conf,\n                    'bbox': [x1, y1, x2, y2],\n                    'detection_id': track_id\n                })\n                 log(f'🚨 LANE: {lp_text}')\n                 \n            tr['prev_positions'][camera_id] = cy\n            \n        result['detections'] = final_detections\n        result['vehicle_count'] = len(final_detections)\n        \n    except Exception as e:\n        log(f'Vehicle Error: {e}')\n        import traceback; traceback.print_exc()\n    \n    return result\n

In [ ]:
# Cell 6: Async WebSocket Client for /ws/kaggle
import websocket
import ssl

class KaggleWebSocket:
    '''WebSocket client for /ws/kaggle path'''
    def __init__(self, ws_url, api_key):
        # Add apiKey to URL
        self.url = f'{ws_url}?apiKey={api_key}'
        self.ws = None
        self.connected = False
        self.send_queue = queue.Queue(maxsize=RESULT_QUEUE_SIZE)
        self._stop = False
    
    def connect(self):
        try:
            log(f'🔌 Connecting to {self.url}')
            self.ws = websocket.create_connection(
                self.url,
                timeout=10,
                sslopt={'cert_reqs': ssl.CERT_NONE}
            )
            self.connected = True
            log('✅ Kaggle WebSocket connected!')
            threading.Thread(target=self._sender_loop, daemon=True).start()
            return True
        except Exception as e:
            log(f'❌ WS Connect failed: {e}')
            return False
    
    def _sender_loop(self):
        while not self._stop:
            try:
                msg = self.send_queue.get(timeout=1)
                if self.ws and self.connected:
                    self.ws.send(json.dumps(msg))
            except queue.Empty:
                continue
            except Exception as e:
                log(f'❌ WS Send error: {e}')
                self.connected = False
    
    def send_async(self, data):
        try:
            self.send_queue.put_nowait(data)
        except queue.Full:
            try:
                self.send_queue.get_nowait()
                self.send_queue.put_nowait(data)
            except: pass

def fetch_cameras():
    '''Fetch camera list and configs from backend (like V15)'''
    global camera_configs, camera_keys
    try:
        resp = requests.get(f'{BACKEND_HTTP_URL}/api/camera/all', timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            cams = data.get('metadata', data) if isinstance(data, dict) else data
            for cam in cams:
                if isinstance(cam, dict) and cam.get('_id'):
                    cam_id = cam['_id']
                    camera_keys[cam_id] = cam.get('camera_api_key', '')
                    camera_configs[cam_id] = {
                        'camera_lane_track_point': cam.get('camera_lane_track_point', []),
                        'camera_lane_vehicles': cam.get('camera_lane_vehicles', [])
                    }
            return list(camera_keys.keys())
    except Exception as e:
        log(f'❌ Fetch cameras error: {e}')
    return []

def test_stream(url, timeout=5):
    try:
        cap = cv2.VideoCapture(url)
        cap.set(cv2.CAP_PROP_OPEN_TIMEOUT_MSEC, timeout * 1000)
        if cap.isOpened():
            ret, _ = cap.read()
            cap.release()
            return ret
        return False
    except: return False

In [ ]:
# Cell 7: Camera Pipeline V16

class CameraPipelineV16:
    def __init__(self, camera_id, api_key, kaggle_ws):
        self.camera_id = camera_id
        self.api_key = api_key
        self.flv_url = f"{NMS_URL}/live/{camera_id}.flv"
        self.frame_queue = queue.Queue(maxsize=FRAME_QUEUE_SIZE)
        self.kaggle_ws = kaggle_ws
        self._stop = False
        self.stats = {'frames_read': 0, 'frames_detected': 0, 'violations': 0, 'fps': 0}
        self.fps_times = deque(maxlen=30)
        self.stream_ready = False
    
    def start(self):
        # Test stream
        log(f'🎥 [{self.camera_id[-4:]}] Testing stream...')
        if not test_stream(self.flv_url, timeout=10):
            log(f'⚠️ [{self.camera_id[-4:]}] Stream not available')
        
        # Start threads
        threading.Thread(target=self._reader_loop, daemon=True).start()
        threading.Thread(target=self._detector_loop, daemon=True).start()
        log(f'🚀 [{self.camera_id[-4:]}] Pipeline started')
        return True
    
    def _reader_loop(self):
        retry_delay = 2
        while not self._stop:
            try:
                cap = cv2.VideoCapture(self.flv_url)
                cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
                
                if not cap.isOpened():
                    time.sleep(retry_delay)
                    retry_delay = min(retry_delay * 2, 30)
                    continue
                
                self.stream_ready = True
                retry_delay = 2
                
                while cap.isOpened() and not self._stop:
                    ret, frame = cap.read()
                    if not ret: break
                    
                    self.stats['frames_read'] += 1
                    try:
                        self.frame_queue.put_nowait(frame)
                    except queue.Full:
                        try:
                            self.frame_queue.get_nowait()
                            self.frame_queue.put_nowait(frame)
                        except: pass
                
                cap.release()
                self.stream_ready = False
            except Exception as e:
                log(f'❌ [{self.camera_id[-4:]}] Reader: {e}')
                time.sleep(retry_delay)
    
    def _detector_loop(self):
        while not self._stop:
            try:
                frame = self.frame_queue.get(timeout=2)
                result = detect_frame(frame, self.camera_id)
                self.stats['frames_detected'] += 1
                
                # Count violations
                if result.get('violations'):
                    self.stats['violations'] += len(result['violations'])
                
                # Send to backend via /ws/kaggle
                self.kaggle_ws.send_async(result)
                
                # FPS
                now = time.time()
                self.fps_times.append(now)
                if len(self.fps_times) > 1:
                    self.stats['fps'] = round(len(self.fps_times) / (self.fps_times[-1] - self.fps_times[0]), 1)
                
                if self.stats['frames_detected'] % 50 == 0:
                    viol = self.stats['violations']
                    tl = result.get('traffic_light', {}).get('status', '-')
                    log(f"[{self.camera_id[-4:]}] {self.stats['fps']}FPS | Det:{len(result.get('detections',[]))} | TL:{tl} | Viol:{viol}")
            except queue.Empty:
                continue
            except Exception as e:
                log(f'❌ [{self.camera_id[-4:]}] Detector: {e}')

In [ ]:
# Cell 8: MAIN - Start V16 Pipelines

log('🔍 Fetching cameras...')
cameras = fetch_cameras()

if not cameras:
    log('❌ No cameras found!')
else:
    log(f'✅ Found {len(cameras)} cameras')
    
    # Create WS connections per camera (using camera_api_key)
    pipelines = []
    for cam_id in cameras:
        cam_key = camera_keys.get(cam_id, '')
        
        # Each camera has its own WS connection with its API key
        kaggle_ws = KaggleWebSocket(BACKEND_WS_URL, cam_key)
        if not kaggle_ws.connect():
            log(f'❌ [{cam_id[-4:]}] WS connect failed, skipping')
            continue
        
        p = CameraPipelineV16(cam_id, cam_key, kaggle_ws)
        if p.start():
            pipelines.append(p)
        time.sleep(0.5)
    
    log(f'🚀 {len(pipelines)} PIPELINES RUNNING!')
    log(f'📡 Using camera API keys from backend')
    
    try:
        while True:
            time.sleep(60)
            total_viol = sum(p.stats['violations'] for p in pipelines)
            for p in pipelines:
                log(f"📊 [{p.camera_id[-4:]}] Read:{p.stats['frames_read']} Det:{p.stats['frames_detected']} FPS:{p.stats['fps']} Viol:{p.stats['violations']} Stream:{'✅' if p.stream_ready else '❌'}")
            log(f'🚨 Total Violations: {total_viol}')
    except KeyboardInterrupt:
        log('Stopping...')